# 하이브리드 - Hybrid Search


In [1]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3
!pip install pickleshare

## 사전준비


### 데이터 준비 및 탐색


In [2]:
import pandas as pd
import requests

df = pd.read_csv("../data/movies.csv", low_memory=False)
df.head(5)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
0,1917,드라마|전쟁,2020,2.19,8.88,5017,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...
1,21 브릿지: 테러 셧다운,범죄|드라마|스릴러,2020,1.01,7.84,579,"뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...",채드윅 보스만|J.K. 시몬스|시에나 밀러|테일러 키취,스테판 제임스|빅토리아 카타게나|키스 데이빗|샤이나 라이언|알렉산더 시디그|모로코 ...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191227_205/1...
2,n번째 이별중,코미디|멜로/로맨스,2020,4.01,6.48,177,여자친구 ‘데비’에게 대차게 차여 충격받은 물리학 천재 ‘스틸먼’. 사랑하는 그녀의...,에이사 버터필드|소피 터너|스카일러 거손도,오브리 레이놀즈|윌 펠츠|질리안 조이|케이든 J. 그레고리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200409_56/15...
3,가슴 큰 태희,멜로/로맨스,2020,2.27,8.67,3,곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 ...,정보없음,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200211_88/15...
4,개인교습,멜로/로맨스,2020,3.02,10.00,1,부잣집 프랑스 아가씨 마고에게 첫눈에 반한 발리의 가난한 어부 에카는 그녀에게 피아...,하리 산티카|도르카스 카핀,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200226_62/15...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11950 entries, 0 to 11949
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       11950 non-null  object 
 1   genre       11950 non-null  object 
 2   year        11950 non-null  int64  
 3   date        11950 non-null  float64
 4   rating      11950 non-null  float64
 5   vote_count  11950 non-null  int64  
 6   plot        11950 non-null  object 
 7   main_act    11950 non-null  object 
 8   supp_act    11950 non-null  object 
 9   page_url    11950 non-null  object 
 10  img_url     11950 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1.0+ MB


In [4]:
df.rename(columns={"plot": "text"}, inplace=True)
df.head(5)

,title,genre,year,date,rating,vote_count,text,main_act,supp_act,page_url,img_url
0,1917,드라마|전쟁,2020,2.19,8.88,5017,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...
1,21 브릿지: 테러 셧다운,범죄|드라마|스릴러,2020,1.01,7.84,579,"뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...",채드윅 보스만|J.K. 시몬스|시에나 밀러|테일러 키취,스테판 제임스|빅토리아 카타게나|키스 데이빗|샤이나 라이언|알렉산더 시디그|모로코 ...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191227_205/1...
2,n번째 이별중,코미디|멜로/로맨스,2020,4.01,6.48,177,여자친구 ‘데비’에게 대차게 차여 충격받은 물리학 천재 ‘스틸먼’. 사랑하는 그녀의...,에이사 버터필드|소피 터너|스카일러 거손도,오브리 레이놀즈|윌 펠츠|질리안 조이|케이든 J. 그레고리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200409_56/15...
3,가슴 큰 태희,멜로/로맨스,2020,2.27,8.67,3,곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 ...,정보없음,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200211_88/15...
4,개인교습,멜로/로맨스,2020,3.02,10.00,1,부잣집 프랑스 아가씨 마고에게 첫눈에 반한 발리의 가난한 어부 에카는 그녀에게 피아...,하리 산티카|도르카스 카핀,정보없음,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200226_62/15...


### OpenSearch 도메인에 연결


In [5]:
# Add system path for utils
import sys

sys.path.insert(0, "/Users/jinhwan/Repository/Labs/opensearch-with-bedrock-workshop-kr/utils")

In [6]:
import boto3, json
from utils import get_cfn_outputs, keyword_search, semantic_search

region_name = "us-east-1"

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]

In [7]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

In [12]:
model_id = "jK-l048BhWoFUAg8WOdO"
index_name = "movie_semantic"

하이브리드 검색 함수


In [13]:
# Refresh the index to make the changes visible
count = aos_client.count(index=index_name)
print(count)

{'count': 11224, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


검색 비교


In [1]:
def keyword_search(query_text):
    query = {
        "size": 10,
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["text"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_id"],
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["text"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_id", "_score", "title", "text", "genre", "rating"]
    )
    display(query_result_df)

In [55]:
query_text = "우주에서 벌어지는 전쟁"

In [56]:
keyword_search(query_text)

,_id,_score,title,text,genre,rating
0,CrAQ1I8BhWoFUAg8pmz2,9.120829,취성의 가르간티아,"우주에서 자라고, 전쟁을 겪으면서 싸우는 법밖에 알지 못하는 소년병 레드는 공간 이...",애니메이션,7.00
1,4bAQ1I8BhWoFUAg8jWnv,7.716440,드라마 스페셜 - 불침번을 서라,아파트에서 연이어 벌어지는 오물이 담긴 쓰레기봉투 투척사건의 범인을 잡기 위해 주민...,드라마,6.33
2,7rAP1I8BhWoFUAg81Fn9,7.561192,콜로니: 인류 최후의 날,우주에서 화물을 나르고 지구로 돌아가던 우주선 오셀롯은 불의의 사고로 소행성에 불시...,모험|SF|스릴러,1.24
3,V7AQ1I8BhWoFUAg8wW6u,7.427664,아이언 스카이,재선을 노리고 있는 최초의 여성 대통령이 통치하고 있는 미합중국에 예기치 않은 소식...,액션|코미디|SF,6.71
4,tLAR1I8BhWoFUAg8D3T-,7.187436,CJ7 - 장강7호,아들만큼은 명문학교에 보내고자 공사장에서 힘들게 일하는 아버지(주성치)와 그의 소중...,코미디|SF|판타지|드라마,7.46
5,iLAR1I8BhWoFUAg8GnWi,7.062734,세토의 신부,"평범한 중학생 나가스미가 어느 여름날 인어 여자아이, 세토썬을 만나면서 벌어지는 이야기.",애니메이션,7.75
6,S7AQ1I8BhWoFUAg8wW6u,7.061657,아름다운 그대에게,남장미소녀가 남자 체고에 위장전학 오면서 벌어지는 좌충우돌 생존기를 그린 드라마.,드라마,7.99
7,F7AP1I8BhWoFUAg8zFla,7.036746,악셀,우주에서 가장 아름다운 행성 케플라. 그러나 지구인들의 욕심으로 행성 케플라는 조금...,애니메이션,4.91
8,cLAQ1I8BhWoFUAg82XBz,6.978060,셜록홈즈 : 그림자 게임,"더욱 강렬하게, 더욱 스릴 있게 세상을 구할 홈즈의 추리가 다시 시작된다! 왓슨과 ...",액션|모험,8.00
9,37AQ1I8BhWoFUAg8jWnv,6.909739,드라마 스페셜 - 당신의 누아르,조폭이 조직의 타깃인 검사의 아내를 사랑하게 되면서 벌어지는 일을 그린 느와르 드라마,드라마,10.00


In [57]:
def semantic_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["vector_field"]},
        "query": {
            "neural": {"vector_field": {"query_text": query_text, "model_id": model_id, "k": 10}},
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_id"],
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["text"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_id", "_score", "title", "plot", "genre", "rating"]
    )
    display(query_result_df)

In [58]:
semantic_search(query_text)

,_id,_score,title,plot,genre,rating
0,mbAQ1I8BhWoFUAg8IWBv,0.665596,인디펜던스 데이: 리써전스,"다시, 그들이 온다! 2 년 전 최악의 우주 전쟁을 치른 지구. 재건을 위해 힘쓴 ...",액션|모험|SF,7.25
1,JLAP1I8BhWoFUAg8tldM,0.654662,하이 라이프,태양계 너머 우주 공간에서 실험 대상이 되기로 받아들인 한 범죄자 무리는 우주선 내...,SF|스릴러,6.56
2,O7AQ1I8BhWoFUAg8P2Pz,0.648699,스페이스 둠스데이,"미래 282 년, 우주를 지배하려는 외계 종족의 위협에 맞서 새로운 삶의 터전을 찾...",판타지|SF,6.20
3,zLAP1I8BhWoFUAg8plVH,0.638531,스페셜 솔져,세계 곳곳에서 전쟁이 동시다발 터지고 존재가 지워진 특수부대가 출동한다. 각지에서 ...,액션|전쟁,6.80
4,_rAQ1I8BhWoFUAg8eGcC,0.630076,인터스텔라,세계 각국의 정부와 경제가 완전히 붕괴된 미래가 다가온다. 지난 2 세기에 범한 잘...,SF,9.10
5,TrAP1I8BhWoFUAg8w1hN,0.624814,몬몬몬 몬스터,인간을 해치는 괴물 괴물을 납치한 인간 친구를 괴롭힌 집단 모든걸 지켜본 당신 누가...,공포|스릴러,7.43
6,lLAQ1I8BhWoFUAg8Cl5j,0.624424,다이버전트 시리즈: 얼리전트,장벽 너머의 새로운 미래에 당도한 다이버전트 군단. 진실을 숨긴 채 인류를 실험대상...,액션|모험|SF,7.28
7,OrAQ1I8BhWoFUAg8hmm3,0.616917,감자별 2013QR3,2 13년 어느 날 지구로 날아온 의문의 행성 '감자별' 때문에 벌어지는 노씨 일가...,드라마,9.49
8,JrAQ1I8BhWoFUAg8hWlS,0.612151,힐러,정치나 사회 정의 같은 건 그저 재수 없는 단어라고 생각하며 살던 청춘들이 부모세대...,드라마,9.27
9,HrAP1I8BhWoFUAg8qlZc,0.611937,어벤져스: 엔드게임,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,액션|SF,9.38


In [65]:
def hybrid_search(query_text):
    query = {
        "size": 10,
        "_source": {"exclude": ["vector_field"]},
        "query": {
            "hybrid": {
                "queries": [
                    {
                        "multi_match": {
                            "query": query_text,
                            "fields": ["title", "text", "genre"],
                        }
                    },
                    {
                        "neural": {
                            "vector_field": {
                                "query_text": query_text,
                                "model_id": model_id,
                                "k": 30,
                            }
                        }
                    },
                ]
            }
        },
        "search_pipeline": {
            "description": "Post processor for hybrid search",
            "phase_results_processors": [
                {
                    "normalization-processor": {
                        "normalization": {"technique": "min_max"},
                        "combination": {
                            "technique": "arithmetic_mean",
                            "parameters": {"weights": [0.3, 0.7]},
                        },
                    }
                }
            ],
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_id"],
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["text"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_id", "_score", "title", "text", "genre", "rating"]
    )
    display(query_result_df)

In [66]:
hybrid_search(query_text)

,_id,_score,title,text,genre,rating
0,mbAQ1I8BhWoFUAg8IWBv,0.700000,인디펜던스 데이: 리써전스,"다시, 그들이 온다! 2 년 전 최악의 우주 전쟁을 치른 지구. 재건을 위해 힘쓴 ...",액션|모험|SF,7.25
1,JLAP1I8BhWoFUAg8tldM,0.625971,하이 라이프,태양계 너머 우주 공간에서 실험 대상이 되기로 받아들인 한 범죄자 무리는 우주선 내...,SF|스릴러,6.56
2,O7AQ1I8BhWoFUAg8P2Pz,0.585604,스페이스 둠스데이,"미래 282 년, 우주를 지배하려는 외계 종족의 위협에 맞서 새로운 삶의 터전을 찾...",판타지|SF,6.20
3,zLAP1I8BhWoFUAg8plVH,0.516764,스페셜 솔져,세계 곳곳에서 전쟁이 동시다발 터지고 존재가 지워진 특수부대가 출동한다. 각지에서 ...,액션|전쟁,6.80
4,_rAQ1I8BhWoFUAg8eGcC,0.459518,인터스텔라,세계 각국의 정부와 경제가 완전히 붕괴된 미래가 다가온다. 지난 2 세기에 범한 잘...,SF,9.10
5,TrAP1I8BhWoFUAg8w1hN,0.423893,몬몬몬 몬스터,인간을 해치는 괴물 괴물을 납치한 인간 친구를 괴롭힌 집단 모든걸 지켜본 당신 누가...,공포|스릴러,7.43
6,lLAQ1I8BhWoFUAg8Cl5j,0.421252,다이버전트 시리즈: 얼리전트,장벽 너머의 새로운 미래에 당도한 다이버전트 군단. 진실을 숨긴 채 인류를 실험대상...,액션|모험|SF,7.28
7,OrAQ1I8BhWoFUAg8hmm3,0.370426,감자별 2013QR3,2 13년 어느 날 지구로 날아온 의문의 행성 '감자별' 때문에 벌어지는 노씨 일가...,드라마,9.49
8,JrAQ1I8BhWoFUAg8hWlS,0.338159,힐러,정치나 사회 정의 같은 건 그저 재수 없는 단어라고 생각하며 살던 청춘들이 부모세대...,드라마,9.27
9,HrAP1I8BhWoFUAg8qlZc,0.336715,어벤져스: 엔드게임,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,액션|SF,9.38


## Clean Up


In [ ]:
aos_client.indices.delete(index=index_name)